In [10]:
import sympy as sp
import string
ascii_greek = ['Α', 'α',
               'Β', 'β',
               'Γ', 'γ',
               'Δ', 'δ',
               'Ε', 'ε',
               'Ζ', 'ζ',
               'Η', 'η',
               'Θ', 'θ',
               'Ι', 'ι',
               'Κ', 'κ',
               'Λ', 'λ',
               'Μ', 'μ',
               'Ν', 'ν',
               'Ξ', 'ξ',
               'Ο', 'ο',
               'Π', 'π',
               'Ρ', 'ρ',
               'Σ', 'σ',
               'Τ', 'τ',
               'Υ', 'υ',
               'Φ', 'φ',
               'Χ', 'χ',
               'Ψ', 'ψ',
               'Ω', 'ω']
latex_greek = ['\\Alpha', '\\alpha',
               '\\Beta', '\\beta',
               '\\Gamma', '\\gamma',
               '\\Delta', '\\delta',
               '\\Epsilon', '\\epsilon',
               '\\Zeta', '\\zeta',
               '\\Eta', '\\eta',
               '\\Theta', '\\theta',
               '\\Iota', '\\iota',
               '\\Kappa', '\\kappa',
               '\\Lambda', '\\lambda',
               '\\Mu', '\\mu',
               '\\Nu', '\\nu',
               '\\Xi', '\\xi',
               '\\Omicron', '\\omicron',
               '\\Pi', '\\pi',
               '\\Rho', '\\rho',
               '\\Sigma', '\\sigma',
               '\\Tau', '\\tau',
               '\\Upsilon', '\\upsilon',
               '\\Phi', '\\phi',
               '\\Chi', '\\chi',
               '\\Psi', '\\psi',
               '\\Omega', '\\omega']

symbols = string.ascii_letters + ''.join(ascii_greek)

def read_file(file_name):
    with open(file_name, 'r') as file:
        for s in file:
            yield s
            
def transform_greek(s: str):
    greek_transform = dict(zip(latex_greek, ascii_greek))
    for letter in latex_greek:
        while letter in s:
            s = s.replace(letter, greek_transform[letter])
    return s

def transform_multiplication(s: str):
    length = len(s)
    i = 1
    while(i < length):
        if (s[i] in symbols and s[i - 1] in (symbols + '1234567890')):
            s = s[:i] + '*' + s[i:]
            length += 1
        i += 1
    return s

def transform_frac(s: str):
    while('\\frac{' in s):
        numerator_begin = s.find('\\frac{')
        numerator_end = s.find('}', numerator_begin)
        denominator_end = s.find('}', numerator_end + 2)
        s = s[:numerator_begin] + s[(numerator_begin + 6):numerator_end] + ' / ' + s[(numerator_end + 2):denominator_end] + s[(denominator_end + 1):]
    return s
        
def transform_formula(s: str):
    if s.find('=') != -1:
        left_part = sp.sympify(s[:s.find('=')])
        right_part = sp.sympify(s[(s.find('=') + 1):])
        return [(left_part, right_part)]
    else:
        equation = sp.sympify(s)
        return [equation]
    
def find_formula(f: str):
    equations = []
    in_equation = 0
    for s in read_file(f):
        if not in_equation:
            if "\\begin{equation}" in s:
                in_equation = 1
            s = transform_greek(s)
            s = transform_frac(s)
            s = transform_multiplication(s)
            if "$$" in s:
                left_index = s.find('$$') + 2
                right_index = s.find('$$', left_index)
                equations += transform_formula(s[left_index:right_index])
            elif "$" in s:
                left_index = s.find('$') + 1
                right_index = s.find('$', left_index)
                equations += transform_formula(s[left_index:right_index])
        elif "\\end{equation}" in s:
            in_equation = 0
        else:
            s = transform_greek(s)
            s = transform_frac(s)
            s = transform_multiplication(s)
            equations += transform_formula(s)
    return equations

In [12]:
equations = find_formula(r"C:\VS Code\Univer\test.tex")
for item in equations:
    if type(item) == tuple:
        print(f'{item[0]} = {item[1]}')
    else:
        print(f'{item}')

13*a - b = f(e**ω)
H*Ψ = E*Ψ
E*q + q(B*υ)
z = a/b + φ
